<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/80x15.png" /></a><div align="center">This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.</div>

----

The ESC403 cluster comes with [Catherine Devlin's `%sql`-magic for IPython][1]; this allows you to run SQL queries from the IPython notebook, and intermix them with Python code.

[1]: https://github.com/catherinedevlin/ipython-sql

Before we can use the `%sql` syntax, two steps must be taken:

* Load the IPython-SQL bridge code

In [1]:
%load_ext sql


* Connect to an actual database; this must be the first `%sql` statement (the funny `rmurri@/lustre` syntax is correct and means "connect to the PostGreSQL DB named `lustre` running on *this* host as user `rmurri`"  -- please replace `rmurri` with your local user name):

In [2]:
%sql postgresql://rmurri@/lustre


u'Connected: rmurri@lustre'

Now you can run 1-line SQL queries by prefixing them with `%sql`:

In [3]:
%sql select * from lustre limit 5;

5 rows affected.


usr,grp,atime,mtime,blksize,size,path
us321,econ,1363100458,1363100620,4,342,/scratch/econ/H2/preprocessing/long.txt
us321,econ,1363100458,1363100620,4,348,/scratch/econ/H2/preprocessing/short.txt
us321,econ,1363100458,1363100620,4,696,/scratch/econ/H2/preprocessing/slist.txt
us321,econ,1363174233,1363174233,16,16135,/scratch/econ/H2/preprocessing/SNS_SSC_008_batch.mat
us321,econ,1363174233,1363174233,16,16142,/scratch/econ/H2/preprocessing/SNS_SSC_009_batch.mat


It is also possible to run multi-line (or multiple) SQL queries by using the `%%sql` syntax instead.  Note that in this case the SQL instructions *must not* be on the same line as the `%%sql` magic marker:

In [ ]:
%%sql
select count(*) from lustre;
select distinct count(usr) from lustre;
select distinct count(grp) from lustre;

The `%%time` magic prints the time taken to evaluate a cell (which comes handy when doing performance comparisons):

In [5]:
%%time

import time
time.sleep(5)

CPU times: user 1.28 ms, sys: 259 µs, total: 1.54 ms
Wall time: 5.01 s


----

**Note:** to keep running times low, we will be using table `lustre_sample` throughout, which contains a sample of 5% the rows of the original `lustre` table.

## 1. How many files are there in the `lustre` dataset

In [7]:
%time %sql select count(*) from lustre_sample;

1 rows affected.
CPU times: user 2.04 ms, sys: 406 µs, total: 2.44 ms
Wall time: 156 ms


count
1519053


Since we know that all values in the `path` column are distinct, we can omit the `DISTINCT` keyword. Does it gain us some performance?

In [8]:
%time %sql select count(distinct path) from lustre_sample;

1 rows affected.
CPU times: user 4.11 ms, sys: 814 µs, total: 4.92 ms
Wall time: 1min 9s


count
1519053


## 2. How many distinct users own files in the dataset?

In [9]:
%sql select count(distinct usr) from lustre_sample;

1 rows affected.


count
40


## 3. What user has the largest number of files?

In [16]:
%sql select distinct usr, count(path) as cnt from lustre_sample group by usr order by cnt desc limit 3;

3 rows affected.


usr,cnt
usr356,602120
usr388,485650
usr264,189651


In [13]:
%sql select sq.usr, sq.count from (select usr, count(path) from lustre_sample group by usr) AS sq order by count desc limit 3;

3 rows affected.


usr,count
usr356,602120
usr388,485650
usr264,189651


## 4. Which is/are the most-recently accessed file(s)?

The `atime` field stores the last-access time in UNIX "epoch" format.
The *maximum* `atime` value is the most recent one::

In [17]:
%sql select max(atime) from lustre_sample;

1 rows affected.


max
1393664299


Then you should of course select the path(s) with that access time::

In [18]:
%sql select path from lustre_sample where atime=1393664299;

1 rows affected.


path
/scratch/bioc/usr384/TD-RS/run08/equi/output/restart.xsc


If we already know the *number* of files with maximum `atime`, we can
use a single query to get their names::

Just in case you wonder: `atime=max(atime)` is not a valid `WHERE` clause::

## 5. Let's say that $T_0$ is the time the most-recently accessed file was last read. How many files were not read in the month preceding $T_0$?

We know that T0=1393671122. Now it's easy to select paths accessed in
the given range::

In [19]:
%sql select count(path) from lustre_sample where atime < (1393664299 - (30*24*3600));

1 rows affected.


count
1492925


## 6. Is it possible to convert fields atime and mtime to PostgreSQL's TIMESTAMP type?

Yes, it *is* possible to alter a SQL table definition after the table has been created.

We shall now show how to (1) create a new table, (2) populate it, then (3) alter the definition and (4) fill the new column with values.

In [ ]:
# (1)+(2) create a new table and populate it


In [ ]:
# (3) alter table definition


In [ ]:
#(4) populate additional column


Show some data from the table we created::

## 7. Can you count the number of files in a given directory?

Yes, using SQL's `like` string matching operator, which allows any
part of a string to be matched by the `%` character (i.e., just like
`*` for file names)::

## 8. Can you find the directory that holds the largest number of files? 

## 9. Can you find the directory tree that holds the largest number of files?

Yes or no? *(and why?)*